# Advanced Spatial Analysis
# Module 09: Spatial weights and ESDA

ESDA: Exploratory Spatial Data Analysis

"Everything is related to everything else, but near things are more related than distant things" -Waldo Tobler

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysal as ps
import seaborn as sns
from scipy.stats import stats

np.random.seed(0)
print(gpd.__version__, ps.__version__)

## Load the tracts data set

In [ ]:
tracts = gpd.read_file('census/census_tracts_data.geojson')
tracts = tracts.set_index('index')
tracts.shape

In [ ]:
tracts.columns

In [ ]:
tracts.head()

In [ ]:
# calculate pop density in persons per sq km
tracts['pop_density'] = tracts['total_pop'] / (tracts['ALAND'] / 1e6)
tracts = tracts.replace([np.inf, -np.inf], np.nan)
tracts = tracts.dropna(subset=['pop_density'])

In [ ]:
utm_ma = '+proj=utm +zone=18 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
tracts = tracts.to_crs(utm_ma)

## 1. Exploratory analysis

In [ ]:
def map_variable(df, col):
    values = df.dropna(subset=[col])
    ax = values.plot(column=col, scheme='quantiles', k=10, cmap='plasma', figsize=(10,10))
    ax.axis('off')
    plt.show()

In [ ]:
map_variable(tracts, 'pop_density')

In [ ]:
map_variable(tracts, 'med_household_income')

It looks like these two variables might be negatively correlated?

In [ ]:
# calculate the correlation coefficient and its p-value
x = tracts.dropna(subset=['pop_density', 'med_household_income'])['pop_density']
y = tracts.dropna(subset=['pop_density', 'med_household_income'])['med_household_income']
r, p = stats.pearsonr(x=x, y=y)
print('r={:.4f}, p={:.4f}'.format(r, p))

In [ ]:
# scatter plot them
fig, ax = plt.subplots()
ax.scatter(x=x, y=y, s=1)
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
plt.show()

In [ ]:
# estimate a simple linear regression model
m, b, r, p, se = stats.linregress(x=x, y=y)
print('m={:.4f}, b={:.4f}, r^2={:.4f}, p={:.4f}'.format(m, b, r ** 2, p))

In [ ]:
# plot the regression line with 95% CI
ax = sns.regplot(x, y, marker='.', scatter_kws={'s':2})
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
plt.show()

## 2. Spatial weights matrix

Spatial weights define the spatial connections among our units of analysis (tracts).

### 2.1. Contiguity-based weights: rook contiguity

Using rook contiguity, two spatial units must share an edge of their boundaries to be considered neighbors. This isn't terribly common in practice (since queen is more useful, but it's worth understanding).

In [ ]:
# get the tract labels (GEOIDs) and pick one to work with later
labels = tracts.index.tolist()
label = labels[900]

In [ ]:
%%time
w_rook = ps.lib.weights.Rook.from_dataframe(tracts)

### 2.2. Contiguity-based weights: queen contiguity

Using queen contiguity, two spatial units need only share a vertex (a single point) of their boundaries to be considered neighbors.

In [ ]:
%%time
w_queen = ps.lib.weights.Queen.from_dataframe(tracts, ids=labels, id_order=labels)

In [ ]:
# find the neighbors of some tract
w_queen.neighbors[label]

In [ ]:
# this is a raw contiguity matrix, so weights are all 1
w_queen.weights[label]

In [ ]:
# how many neighbors does this tract have?
w_queen.cardinalities[label]

In [ ]:
# convert cardinalites to series and describe data -- looks right-skewed
cardinalites_queen = pd.Series(w_queen.cardinalities)
cardinalites_queen.describe()

In [ ]:
ax = cardinalites_queen.hist(bins=20)

In [ ]:
# number of observations
w_queen.n

In [ ]:
# average number of neighbors
w_queen.mean_neighbors

In [ ]:
# min number of neighbors
w_queen.min_neighbors

In [ ]:
# max number of neighbors
w_queen.max_neighbors

In [ ]:
# islands (observations with no neighbors, disconnected in space)
w_queen.islands

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
tracts.plot(ax=ax, facecolor='#999999', edgecolor='w', linewidth=0.5)

# plot some tract of interest in red
tract = tracts.loc[[label]]
tract.plot(ax=ax, facecolor='r', edgecolor='w', linewidth=1)

# plot the neighbors in blue
neighbors = tracts.loc[w_queen[label]]
neighbors.plot(ax=ax, facecolor='b', edgecolor='w', linewidth=1)

# zoom to area of interest
xmin, ymin, xmax, ymax = neighbors.unary_union.bounds
ax.axis('equal')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_title('Neighbors of tract {}'.format(label))
plt.show()

In [ ]:
%%time
# draw a contiguity graph of the tracts
fig, ax = plt.subplots(figsize=(12, 12))
tracts.plot(ax=ax, facecolor='#333333', edgecolor='w', linewidth=0.2)

for label, neighbors in w_queen:
    focal = np.hstack(tracts.loc[label, 'geometry'].centroid.xy)
    centroids = np.vstack(tracts.loc[neighbors, 'geometry'].apply(lambda g: (g.centroid.x, g.centroid.y)).values)
    for neighbor in centroids:
        ax.plot(*zip(focal, neighbor), color='r', linewidth=0.3)

ax.axis('off')
plt.show()

### 3.3. Distance-based weights: k-nn

Find the k-nearest neighbors of each tract

In [ ]:
# k-nearest neighbors finds the closest k tract centroids to each tract centroid
# here, k=6
w_knn = ps.lib.weights.KNN.from_dataframe(tracts, k=6)

### 3.4. Distance-based weights: distance band

Tracts are considered neighbors of some tract if they are within some threshold distance of it.

In [ ]:
x = tracts.centroid.x
y = tracts.centroid.y
coords = np.array([x, y]).T
threshold = ps.lib.weights.min_threshold_distance(coords)

In [ ]:
%%time
w_dist = ps.lib.weights.distance.DistanceBand.from_dataframe(tracts, threshold=threshold)

In [ ]:
len(w_dist.neighbors[label])

### 3.5. Standardizing weights

A spatial weights matrix with raw values (e.g. 1s and 0s for neighbor/not) is not always the best for analysis. Some sort of standardization is useful.

In [ ]:
w_queen[label]

In [ ]:
# check the current transformation of the weights matrix (O = original)
w_queen.transform

Typically, we want to apply a row-based transformation, so every row of the matrix sums up to 1.

In [ ]:
w_queen.transform = 'R'
w_queen[label]

PySAL supports the following transformations:

  - O: original, returning the object to the initial state
  - B: binary, with every neighbor having assigned a weight of 1
  - R: row-based, with all the neighbors of a given observation adding up to 1
  - V: variance stabilizing, with the sum of all the weights being constrained to the number of observations

**It can take a long time to calculate a weights matrix for a large data set.**

Once you've created yours, you might want to save it to disk to re-use in subsequent analyses.

In [ ]:
# save your matrix to disk
f = ps.lib.io.open('tracts_queen.gal', 'w')
f.write(w_queen)
f.close()

In [ ]:
# read a matrix from disk (notice its transformation)
w_queen = ps.lib.io.open('tracts_queen.gal', 'r').read()
w_queen[label]

## 4. Spatial lag

Using the `med_household_income` variable. If the spatial weights matrix is row-standardized, then the spatial lag is the average value of an observation's neighbors.

In [ ]:
col = 'med_household_income'
tracts_not_null = tracts[[col, 'geometry']].dropna()
y = tracts_not_null[col]

In [ ]:
w_queen = ps.lib.weights.Queen.from_dataframe(tracts_not_null)
w_queen.transform = 'R'

In [ ]:
# compute spatial lag
y_lag = ps.lib.weights.lag_spatial(w_queen, y)

In [ ]:
col_lag = '{}_lag'.format(col)
data_lag = pd.DataFrame(data={col:y, col_lag:y_lag}).astype(int)
data_lag.sample(10)

## 6. Spatial autocorrelation

Statistical models typically assume that the observations are independent of each other. This assumption is violated when a variable's value at one location is correlated with its value at nearby locations. This is called spatial autocorrelation, and is common in the real world due to proximity-based spillover effects. Substantive spatial autocorrelation can be explained by social or economic theory that describes a spatial relationship. Nuisance spatial autocorrelation stems from data problems.

*Positive spatial autocorrelation*: nearby values tend to be more similar (e.g. income, home values, temperature, rainfall)

*Negative spatial autocorrelation*: nearby values tend to be more dissimilar (e.g. fire stations, grocery stores)

### 6.1. Moran's I

Moran's I measures *global* spatial autocorrelation: do values tend to be near other (dis)similar values. Values > 1 indicate positive spatial autocorrelation, and values < 1 indicate negative spatial autocorrelation.

In [ ]:
# calculate the statistic
mi = ps.explore.esda.Moran(data_lag[col], w_queen)

In [ ]:
# show the I value
mi.I

In [ ]:
# statistical inference: show the p value
mi.p_sim

If we generated a large number of maps with the same values but randomly allocated over space, and calculated Moran's I for each of these maps, only 0.1% of them would display a larger absolute value than the one we computed from the real-world data set. Thus there is a 0.1% chance of getting the observed value of Moran's I if the spatial distribution of our variable is random. We can conclude that the variable's distribution is statistically-significantly postively spatially autocorrelated.

### 6.2. Moran plots

A Moran plot scatter plots the spatially-lagged values (y-axis) vs the original variable's values (x-axis). Moran's I is the slope of the line in a Moran plot, which makes this a bit easier to conceptualize.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
sns.regplot(x=col, y=col_lag, data=data_lag, scatter_kws={'s':1})
plt.show()

In [ ]:
# standardize the vector (i.e., calculate z-scores)
y_std = (y - y.mean()) / y.std()

In [ ]:
# compute the spatial lag of the standardized vector and save it as a series indexed like the original vector
y_std_lag = pd.Series(ps.lib.weights.lag_spatial(w_queen, y_std), index=y_std.index, name=col_lag)

In [ ]:
# standardized moran's plot, ignoring outliers beyond 3 std devs
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
plt.axvline(0, c='k', alpha=0.5)
plt.axhline(0, c='k', alpha=0.5)
sns.regplot(ax=ax, x=y_std, y=y_std_lag, scatter_kws={'s':1})
plt.show()

Notice the 95% confidence interval shading and the positive slope. Given the value of Moran's I that we calculated earlier (and its p-value), we can conclude that the slope of the line is statistically-significantly different from zero.

In [ ]:
# estimate a simple linear regression model
m, b, r, p, se = stats.linregress(x=y_std, y=y_std_lag)
print('m={:.4f}, b={:.4f}, r^2={:.4f}, p={:.4f}'.format(m, b, r ** 2, p))

In [ ]:
# the slope is the same as moran's I
mi.I

### 6.3. LISAs

Local Indicators of Spatial Autocorrelation: are there specific areas with high concentrations of (dis)similar values?

Moran's I tells us about spatial clustering across the data set as a whole. However, it does not tell us where these clusters occur. For that, we need a local measure. Essentially, we will classify the data set's observations into four groups based on the four quadrants of the Moran plot:

  1. HH: high value near other high values (hot spots)
  1. LL: low value near other low values (cold spots)
  1. HL: high value near low values (spatial outliers)
  1. LH: low value near high values (spatial outliers)

In [ ]:
# standardized moran's plot again, from subsection above
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
plt.axvline(0, c='k', alpha=0.5)
plt.axhline(0, c='k', alpha=0.5)
ax.text(1.25, 1.25, 'HH', fontsize=20)
ax.text(1.25, -1.75, 'HL', fontsize=20)
ax.text(-1.75, 1.25, 'LH', fontsize=20)
ax.text(-1.75, -1.75, 'LL', fontsize=20)
sns.regplot(ax=ax, x=y_std, y=y_std_lag, scatter_kws={'s':1})
plt.show()

In [ ]:
lisa = ps.explore.esda.Moran_Local(data_lag[col], w_queen)

In [ ]:
# set the significance threshold
alpha = 0.05

In [ ]:
# identify whether each observation is significant or not
# p-value interpretation same as earlier with moran's I
data_lag['significant'] = lisa.p_sim < alpha
data_lag['significant'].value_counts()

In [ ]:
# identify the quadrant each observation belongs to
data_lag['quadrant'] = lisa.q
data_lag['quadrant'] = data_lag['quadrant'].replace({1:'HH', 2:'LH', 3:'LL', 4:'HL'})
data_lag['quadrant'].sort_values().value_counts()

In [ ]:
# merge the original tracts and LISA quadrants data together
tracts_lisa = gpd.GeoDataFrame(pd.merge(tracts, data_lag, how='left', left_index=True, right_index=True))

In [ ]:
# create figure and axis then draw the basemap of tracts
fig, ax = plt.subplots(figsize=(9, 9))
tracts_lisa.plot(ax=ax, facecolor='#999999', edgecolor='k', linewidth=0.2)

# plot each quandrant's tracts (if significant LISA statistic) in a different color
quadrant_colors = {'HH':'r', 'LL':'b', 'LH':'skyblue', 'HL':'pink'}
for q, c in quadrant_colors.items():
    mask = tracts_lisa['significant'] & (tracts_lisa['quadrant'] == q)
    rows = tracts_lisa.loc[mask]
    rows.plot(ax=ax, color=c, edgecolor='k', linewidth=0.2)

ax.set_axis_off()
plt.show()

In red we see clusters of tracts with high values surrounded by other high values. In blue we see clusters of tracts with low values surrounded by other low values. In pink, we see the first type of spatial outliers: tracts with high values but surrounded by low values. Finally, in light blue we see the other type of spatial outlier: tracts with low values surrounded by other tracts with high values.